In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 각 gpkg 파일은 분리되어 있는것 같다. 합쳐주기 위해서 folder path를 받아오자 
old_f_path = f'/workspace/daegu/data/external_open/대구 빅데이터 마트 데이터/7. 안전/2. 보행노인사고 다발지역'
jaywalk_f_path = f'/workspace/daegu/data/external_open/대구 빅데이터 마트 데이터/7. 안전/5. 보행자무단횡단사고 다발지역'
ice_f_path = f'/workspace/daegu/data/external_open/대구 빅데이터 마트 데이터/7. 안전/8. 결빙사고 다발지역'
truck_f_path = f'/workspace/daegu/data/external_open/대구 빅데이터 마트 데이터/7. 안전/9. 화물차사고 다발지역'
walker_f_path = f'/workspace/daegu/data/external_open/대구 빅데이터 마트 데이터/7. 안전/11. 보행자사고 다발지역'

# 각 folder path 내에서 gpkg 확장자 파일의 이름을 추출해서 list로 만들자 
old_fnames = glob.glob(f'{old_f_path}/*.gpkg')
jaywalk_fnames = glob.glob(f'{jaywalk_f_path}/*.gpkg')
ice_fnames = glob.glob(f'{ice_f_path}/*.gpkg')
truck_fnames = glob.glob(f'{truck_f_path}/*.gpkg')
walker_fnames = glob.glob(f'{walker_f_path}/*.gpkg')

# fnames 변수의 list를 만들자 
fname_list = [old_fnames, jaywalk_fnames, ice_fnames, truck_fnames, walker_fnames]

In [ ]:
import geopandas as gpd

def gpd_merge(file_names):

    gdfs = []

    # Load each GeoPackage file in the list
    for file_name in file_names:
        try:
            gdf = gpd.read_file(file_name, encoding='cp949')
            
            gdf = gdf.dropna()

            gdfs.append(gdf)

            print(f"Loaded GeoPackage file: {file_name}")
            print(f"Number of rows after removing missing values: {len(gdf)}")
        except Exception as e:
            print(f"Error loading file {file_name}: {e}")

    # 좌표계 변환 : EPSG:5179 -> EPSG:4326
    merged_gdf = pd.concat(gdfs, axis=0)
    merged_gdf.geometry = merged_gdf.geometry.to_crs('EPSG:4326')

    # geometry 열로부터 위도, 경도 열을 추가함 (multipoligon의 center?)
    merged_gdf['위도'] = merged_gdf['geometry'].apply(lambda geom: geom.centroid.y if geom.geom_type == 'Point' else geom.centroid.y)
    merged_gdf['경도'] = merged_gdf['geometry'].apply(lambda geom: geom.centroid.x if geom.geom_type == 'Point' else geom.centroid.x)

    return merged_gdf

In [ ]:
old_df = gpd_merge(old_fnames)
jaywalk_df = gpd_merge(jaywalk_fnames)
ice_df = gpd_merge(ice_fnames)
truck_df = gpd_merge(truck_fnames)
walker_df = gpd_merge(walker_fnames)

In [ ]:
PATH = f'/workspace/daegu/data/external_open'

old_df.to_csv(os.path.join(PATH, "보행노인사고.csv"), encoding='cp949', index=False)
jaywalk_df.to_csv(os.path.join(PATH, "무단횡단사고.csv"), encoding='cp949', index=False)
ice_df.to_csv(os.path.join(PATH, "결빙사고.csv"), encoding='cp949', index=False)
truck_df.to_csv(os.path.join(PATH, "화물차사고.csv"), encoding='cp949', index=False)
walker_df.to_csv(os.path.join(PATH, "보행자사고.csv"), encoding='cp949', index=False)

In [ ]:
gdf_list = [old_df, jaywalk_df, ice_df, truck_df, walker_df]
# 열이름이 동일하다 (다발지역내 사고 / 사상자 / 사망자 / 중상자 / 경상자 / 부상자 / 경상자 / 부상신고자수 합계 및 평균을 의미), TAAS API의 설명 참조
# occrrnc_cnt : 사고건수 / caslt_cnt : 사상자수 / dth_dnv_cnt : 사망자수 / se_dnv_cnt : 중상자수 / sl_dnv_cnt : 경상자수 / wnd_dnv_cnt : 부상신고자수
#  

for gdf in gdf_list:
    print(len(gdf), gdf.columns.to_list())

## 구 예측

In [ ]:
# TRAIN, TEST 데이터 전처리 함수 
def convert_df(df):
# 1. 사고일시, 요일 열 전처리 하는 함수 

    # 사고일시를 datetime 형태로 변환
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    
    # 사고일시로부터 연/월/일/시 열 생성
    df['year'] = df['사고일시'].dt.year
    df['month'] = df['사고일시'].dt.month
    df['day'] = df['사고일시'].dt.day
    df['hour'] = df['사고일시'].dt.hour

    # '사고일시' 로부터 요일 category형으로 label encoding 
    df['요일'] = df['사고일시'].dt.day_of_week.astype('category')
    # 요일에서 '월'만 남기기
    # df['요일'] = df['요일'].str.replace('요일','')   

# 2. 시군구 -> 시/군/구 구분
    df['시'] = df['시군구'].str.split(' ').str.get(0)
    df['군'] = df['시군구'].str.split(' ').str.get(1)
    df['구'] = df['시군구'].str.split(' ').str.get(2)

# 3. 도로형태 -> 도로형태_대 / 도로형태_중 으로 구분
    df['도로형태_대'] = df['도로형태'].str.split(' - ').str.get(0)
    df['도로형태_중'] = df['도로형태'].str.split(' - ').str.get(1)
    
    return df

In [ ]:
PATH = '/workspace/daegu/data'

train_df = pd.read_csv(f'{PATH}/train.csv')
test_df = pd.read_csv(f'{PATH}/test.csv')

In [ ]:
train_df = convert_df(train_df)
test_df = convert_df(test_df)

gu_list = train_df['구'].unique().tolist()

In [ ]:
PATH_EXT = '/workspace/daegu/data/external_open'

seculight_df = pd.read_csv(f"{PATH_EXT}/대구 보안등 정보.csv", encoding= 'cp949')
child_df = pd.read_csv(f"{PATH_EXT}/대구 어린이 보호 구역 정보.csv", encoding= 'cp949')
parking_df = pd.read_csv(f"{PATH_EXT}/대구 주차장 정보.csv", encoding= 'cp949')
cctv_df = pd.read_csv(f"{PATH_EXT}/대구 CCTV 정보.csv", encoding= 'cp949')

df_dict = {'보안등':seculight_df, '어린이보호구역':child_df, '주차장':parking_df, 'cctv':cctv_df}

In [ ]:
# df와 unique list(A:구)를 입력해서 '구' 값을 추출해서 열을 새로 만드는 전처리 함수
def preprocess_df(df, A):
    # 1) Create a new column '구' to store the values
    df['구'] = np.nan

    # 2) Iterate through each row in the data frame
    for index, row in df.iterrows():
        # Check if the value in '소재지지번주소' is not NaN
        if not pd.isna(row['소재지지번주소']):
            # Check if any value in A is present in the '소재지지번주소' column
            for value in A:
                if value in row['소재지지번주소']:
                    # If found, store the value in column '구'
                    df.at[index, '구'] = value
                    break  # Break the loop if a match is found     
    
    return df

In [ ]:
for key, df in df_dict.items():
    print(key, '|열 개수:',  len(df.columns), '|열 이름:', df.columns.tolist())

In [ ]:
for key, df in df_dict.items():
     df = preprocess_df(df, gu_list)

In [ ]:
for key, df in df_dict.items():
    print(key, '|열 개수:',  len(df.columns), '|열 이름:', df.columns.tolist())

In [ ]:
seculight_df.to_csv(f"{PATH_EXT}/대구 보안등 정보_구추가.csv", encoding= 'cp949', index=False)
child_df.to_csv(f"{PATH_EXT}/대구 어린이 보호 구역 정보_구추가.csv", encoding= 'cp949', index=False)
parking_df.to_csv(f"{PATH_EXT}/대구 주차장 정보_구추가.csv", encoding= 'cp949', index=False)
cctv_df.to_csv(f"{PATH_EXT}/대구 CCTV 정보_구추가.csv", encoding= 'cp949', index=False)

In [ ]:
gu_df = pd.concat(df_dict.values(), axis=0)[['위도', '경도', '구']]

In [ ]:
gu_df = gu_df.dropna()

X = gu_df[['위도', '경도']]
y = gu_df['구']

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 909
seed_everything(SEED)

In [ ]:
from supervised.automl import AutoML

model = AutoML(
    mode = 'Compete',
    ml_task = 'multiclass_classification',
    algorithms = ['LightGBM', 'CatBoost', 'Xgboost'],
    n_jobs = -1,
    random_state = SEED,
)

In [34]:
model.fit(X, y)

[43]	train's multi_logloss: 29.2247	validation's multi_logloss: 29.1973
[44]	train's multi_logloss: 29.0253	validation's multi_logloss: 29.0003
[45]	train's multi_logloss: 30.1667	validation's multi_logloss: 30.364
[46]	train's multi_logloss: 30.0096	validation's multi_logloss: 30.2011
[47]	train's multi_logloss: 29.9975	validation's multi_logloss: 30.2011
[48]	train's multi_logloss: 30.0265	validation's multi_logloss: 30.1934
[49]	train's multi_logloss: 29.9047	validation's multi_logloss: 30.1002
[50]	train's multi_logloss: 29.9018	validation's multi_logloss: 30.0924
[51]	train's multi_logloss: 29.751	validation's multi_logloss: 29.9462
2_Default_LightGBM logloss 1.333145 trained in 35.7 seconds
3_Default_Xgboost logloss 0.103263 trained in 54.54 seconds
4_Default_CatBoost logloss 0.135755 trained in 387.75 seconds
* Step not_so_random will try to check up to 27 models
[1]	train's multi_logloss: 0.956523	validation's multi_logloss: 1.12109
[2]	train's multi_logloss: 7.74298	validation

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.


None 10
Add Golden Feature: 경도_sum_위도
Add Golden Feature: 경도_multiply_위도
Add Golden Feature: 경도_ratio_위도
Add Golden Feature: 위도_ratio_경도
Add Golden Feature: 위도_diff_경도
Created 5 Golden Features in 6.61 seconds.
3_Default_Xgboost_GoldenFeatures logloss 0.074247 trained in 94.09 seconds
6_Xgboost_GoldenFeatures logloss 0.068545 trained in 94.41 seconds
* Step kmeans_features will try to check up to 3 models
3_Default_Xgboost_KMeansFeatures logloss 0.084171 trained in 89.64 seconds
* Step insert_random_feature will try to check up to 1 model


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do n

6_Xgboost_GoldenFeatures_RandomFeature logloss 0.080326 trained in 107.67 seconds
Drop features ['random_feature']
Skip features_selection because no parameters were generated.
* Step hill_climbing_1 will try to check up to 16 models
25_Xgboost_GoldenFeatures logloss 0.067554 trained in 252.65 seconds
26_Xgboost_GoldenFeatures logloss 0.06924 trained in 256.58 seconds
* Step hill_climbing_2 will try to check up to 17 models
27_Xgboost_GoldenFeatures logloss 0.065223 trained in 343.04 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.05889 trained in 19.81 seconds
AutoML fit time: 3754.48 seconds
AutoML best model: Ensemble


AutoML(algorithms=['LightGBM', 'CatBoost', 'Xgboost'],
       ml_task='multiclass_classification', mode='Compete', random_state=909)

In [35]:
# 저장 및 불러오기 
from joblib import dump, load

model_filename = 'gu_model_LCX.joblib'
dump(model, model_filename)

gu_model = load(model_filename)

In [36]:
# gu model로 gpd에서 '구'를 예측하자 
for gdf in gdf_list:
    gdf['구'] = gu_model.predict(gdf[['위도', '경도']])

    # 사용안할 열을 drop
    cols_remove = ['geometry', 'occrrnc_cnt_mean', 'caslt_cnt_mean', 'dth_dnv_cnt_mean', 'se_dnv_cnt_mean', 'sl_dnv_cnt_mean', 'wnd_dnv_cnt_mean', '위도', '경도', 'id']

    if all(column in gdf.columns for column in cols_remove):
        gdf.drop(cols_remove, axis=1, inplace=True)
    # gdf.reset_index(drop=True, inplace=True)

In [37]:
# 각 df로부터 구 기준으로 집계된 새로운 df들을 만들자 (agg_dfs에 저장)
def create_agg_df(dataframes):

    aggregated_dfs = []

    for df in dataframes:
        # Perform groupby operation and aggregate based on the specified column
        aggregated_df = df.groupby('구').sum().reset_index()
        aggregated_dfs.append(aggregated_df)

    return aggregated_dfs

# Example: Create aggregated data frames based on the 'old' column
agg_dfs = create_agg_df(gdf_list)

In [38]:
agg_dfs[-1].head()

,구,count,occrrnc_cnt_sum,caslt_cnt_sum,dth_dnv_cnt_sum,se_dnv_cnt_sum,sl_dnv_cnt_sum,wnd_dnv_cnt_sum
0,감삼동,15.0,72.0,96.0,3.0,72.0,21.0,0.0
1,공평동,1.0,4.0,6.0,0.0,4.0,2.0,0.0
2,교동,2.0,8.0,12.0,0.0,8.0,4.0,0.0
3,구암동,1.0,4.0,4.0,0.0,4.0,0.0,0.0
4,남산동,9.0,36.0,69.0,0.0,36.0,28.0,5.0


In [39]:
# train 데이터와 합쳐주기 위해서 각 집계된 리스트의 열이름을 구별해서 합쳐주자 
string_list = ['old', 'jay', 'ice', 'truck', 'walker']

# Iterate over each data frame in the list and modify column names
for i, (agg_df, prefix) in enumerate(zip(agg_dfs, string_list)):
    if prefix != '구':
        # Modify column names based on the rules
        new_column_names = [prefix + '_' + col if col != '구' else col for col in agg_df.columns]
        agg_df.columns = new_column_names

In [40]:
for df in agg_dfs:
    print(df.columns.to_list() )
    # print(len(df), len(df['구'].unique()))

['구', 'old_count', 'old_occrrnc_cnt_sum', 'old_caslt_cnt_sum', 'old_dth_dnv_cnt_sum', 'old_se_dnv_cnt_sum', 'old_sl_dnv_cnt_sum', 'old_wnd_dnv_cnt_sum']
['구', 'jay_count', 'jay_occrrnc_cnt_sum', 'jay_caslt_cnt_sum', 'jay_dth_dnv_cnt_sum', 'jay_se_dnv_cnt_sum', 'jay_sl_dnv_cnt_sum', 'jay_wnd_dnv_cnt_sum']
['구', 'ice_count', 'ice_occrrnc_cnt_sum', 'ice_caslt_cnt_sum', 'ice_dth_dnv_cnt_sum', 'ice_se_dnv_cnt_sum', 'ice_sl_dnv_cnt_sum', 'ice_wnd_dnv_cnt_sum']
['구', 'truck_count', 'truck_occrrnc_cnt_sum', 'truck_caslt_cnt_sum', 'truck_dth_dnv_cnt_sum', 'truck_se_dnv_cnt_sum', 'truck_sl_dnv_cnt_sum', 'truck_wnd_dnv_cnt_sum']
['구', 'walker_count', 'walker_occrrnc_cnt_sum', 'walker_caslt_cnt_sum', 'walker_dth_dnv_cnt_sum', 'walker_se_dnv_cnt_sum', 'walker_sl_dnv_cnt_sum', 'walker_wnd_dnv_cnt_sum']


In [41]:
from functools import reduce

agg_merged_df = reduce(lambda left, right: pd.merge(left, right, on='구', how='outer'), agg_dfs)
agg_merged_df = agg_merged_df.fillna(0)
print(agg_merged_df.shape, agg_merged_df.columns.to_list())

(92, 36) ['구', 'old_count', 'old_occrrnc_cnt_sum', 'old_caslt_cnt_sum', 'old_dth_dnv_cnt_sum', 'old_se_dnv_cnt_sum', 'old_sl_dnv_cnt_sum', 'old_wnd_dnv_cnt_sum', 'jay_count', 'jay_occrrnc_cnt_sum', 'jay_caslt_cnt_sum', 'jay_dth_dnv_cnt_sum', 'jay_se_dnv_cnt_sum', 'jay_sl_dnv_cnt_sum', 'jay_wnd_dnv_cnt_sum', 'ice_count', 'ice_occrrnc_cnt_sum', 'ice_caslt_cnt_sum', 'ice_dth_dnv_cnt_sum', 'ice_se_dnv_cnt_sum', 'ice_sl_dnv_cnt_sum', 'ice_wnd_dnv_cnt_sum', 'truck_count', 'truck_occrrnc_cnt_sum', 'truck_caslt_cnt_sum', 'truck_dth_dnv_cnt_sum', 'truck_se_dnv_cnt_sum', 'truck_sl_dnv_cnt_sum', 'truck_wnd_dnv_cnt_sum', 'walker_count', 'walker_occrrnc_cnt_sum', 'walker_caslt_cnt_sum', 'walker_dth_dnv_cnt_sum', 'walker_se_dnv_cnt_sum', 'walker_sl_dnv_cnt_sum', 'walker_wnd_dnv_cnt_sum']


In [42]:
train_df = pd.merge(train_df, agg_merged_df, on='구', how='left').fillna(0)
test_df = pd.merge(test_df, agg_merged_df, on='구', how='left').fillna(0)
print(train_df.shape, test_df.shape)

(39609, 67) (10963, 52)


In [43]:
agg_sec_df = seculight_df[['구', '설치개수']].groupby(['구']).sum().reset_index()
agg_sec_df.columns = ['구', '보안등_수']
agg_sec_df.head()

,구,보안등_수
0,가창면,1123
1,각산동,139
2,갈산동,351
3,감삼동,941
4,검단동,391


In [44]:
child_df['count'] = 1
agg_child_df = child_df[['구', 'CCTV설치대수', 'count']].groupby('구').sum().reset_index()
agg_child_df.columns = ['구', '어린이_CCTV_설치대수', '어린이구역_수']
agg_child_df.head()

,구,어린이_CCTV_설치대수,어린이구역_수
0,가창면,0.0,8
1,검단동,4.0,1
2,고성동1가,2.0,1
3,고성동2가,2.0,1
4,관음동,22.0,5


In [45]:
parking_df['count'] = 1
agg_parking_df = parking_df[['구', 'count']].groupby('구').sum().reset_index()
agg_parking_df.columns = ['구', '주차장_수']
agg_parking_df.head()

,구,주차장_수
0,가창면,2
1,갈산동,4
2,감삼동,4
3,검단동,1
4,계산동1가,2


In [46]:
agg_csv_dfs = [agg_sec_df, agg_child_df, agg_parking_df]

agg_csv_df = reduce(lambda left, right: pd.merge(left, right, on='구', how='outer'), agg_csv_dfs)
agg_csv_df = agg_csv_df.fillna(0)
print(agg_csv_df.shape, len(agg_csv_df['구'].unique()), agg_csv_df.columns.to_list())

(182, 5) 182 ['구', '보안등_수', '어린이_CCTV_설치대수', '어린이구역_수', '주차장_수']


In [47]:
train_df = pd.merge(train_df, agg_csv_df, on='구', how='left').fillna(0)
test_df = pd.merge(test_df, agg_csv_df, on='구', how='left').fillna(0)
print(train_df.shape, test_df.shape)

(39609, 71) (10963, 56)


In [48]:
train_df.to_csv(f"train_new.csv", encoding= 'cp949', index=False)
test_df.to_csv(f"test_new.csv", encoding= 'cp949', index=False)